In [1]:
## imporrt libarary 
import os
import pandas as pd

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
# from google.cloud import aiplatform
# from support_functions import missing_value, fill_missing, list_dtypes

## sklearn module
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import pickle


In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "trial_bigq.json"

In [3]:
project_id = 'dtidsUS'
dataset_id = 'capstone'
table_id = 'data_telco_customer_churn'
region = 'us-central1'
bucket_name = 'modul4'
blob_name = 'm_fahd/data_telco_customer_churn.csv'

client = bigquery.Client(project='dtidsus')

In [4]:
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name)
    data_capstone = bucket.blob('fahd/data_telco_customer_churn.csv')
    data_capstone.upload_from_filename('data_telco_customer_churn.csv')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


In [5]:
client = bigquery.Client('dtidsus')

In [6]:
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")

In [7]:
df = query_job.result().to_dataframe()

/opt/anaconda3/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [9]:




#cleansing 
# result = df.drop(['ID'], axis = 1)
df.head()

,Dependents,tenure,OnlineSecurity,OnlineBackup,InternetService,DeviceProtection,TechSupport,Contract,PaperlessBilling,MonthlyCharges,Churn
0,False,49,No internet service,No internet service,No,No internet service,No internet service,One year,False,19.0,False
1,True,13,No internet service,No internet service,No,No internet service,No internet service,One year,False,20.0,False
2,False,11,No internet service,No internet service,No,No internet service,No internet service,One year,False,20.0,False
3,True,11,No internet service,No internet service,No,No internet service,No internet service,One year,True,20.0,False
4,False,36,No internet service,No internet service,No,No internet service,No internet service,One year,False,20.0,False


In [10]:
df.isna().sum()

Dependents          0
tenure              0
OnlineSecurity      0
OnlineBackup        0
InternetService     0
DeviceProtection    0
TechSupport         0
Contract            0
PaperlessBilling    0
MonthlyCharges      0
Churn               0
dtype: int64

In [11]:
result  = df.dropna()

In [12]:
categorical_cols = result.select_dtypes(include=['object']).columns

# Create a dictionary to store label encoders for each column
encoders = {}

# Iterate through categorical columns
for col in categorical_cols:
    # Create a LabelEncoder object for each column
    encoder = LabelEncoder()

    # Fit and transform the column
    result[col] = encoder.fit_transform(result[col])

    # Store the encoder in the dictionary for later use
    encoders[col] = encoder


# Preprocessing
y = result['Churn']  
X = result.drop(['Churn'], axis=1)  

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Evaluation
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

# Save Model
model_filename = "model.pkl"
pickle.dump(model, open(model_filename, 'wb'))

              precision    recall  f1-score   support

         0.0       0.83      0.89      0.86       726
         1.0       0.61      0.48      0.54       260

    accuracy                           0.78       986
   macro avg       0.72      0.69      0.70       986
weighted avg       0.77      0.78      0.77       986



/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##upload the model

In [13]:
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob('m_fahd/model/model.pkl')
    blob_model.upload_from_filename('model.pkl')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


In [16]:
from google.cloud import aiplatform

In [17]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "dev_trial.json"

In [18]:
aiplatform.init(project='dtidsus', location='us-central1')

model = aiplatform.Model.upload(
    display_name='fahd_model_000',
    artifact_uri="gs://modul4/m_fahd/model/",
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-2:latest",
)

model.wait()

Creating Model
Create Model backing LRO: projects/41965541199/locations/us-central1/models/724187836076523520/operations/2968245335555571712
Model created. Resource name: projects/41965541199/locations/us-central1/models/724187836076523520@1
To use this Model in another session:
model = aiplatform.Model('projects/41965541199/locations/us-central1/models/724187836076523520@1')


In [ ]:
# endpoint = aiplatform.Endpoint.create(
#     display_name="jaya-endpoint-000",
#     project='dtidsus',
#     location='us-central1',
# )

Creating Endpoint
Create Endpoint backing LRO: projects/41965541199/locations/us-central1/endpoints/8783753203209469952/operations/8980550838095183872
Endpoint created. Resource name: projects/41965541199/locations/us-central1/endpoints/8783753203209469952
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/41965541199/locations/us-central1/endpoints/8783753203209469952')


In [21]:
min_replica_count: int = 1
max_replica_count: int = 1

In [24]:
endpoint.deploy( 
    model=model,
    deployed_model_display_name='fahd_model_000',
    machine_type='e2-standard-2',
    min_replica_count=min_replica_count,
    max_replica_count=max_replica_count,
    sync=True, 
)

Deploying Model projects/41965541199/locations/us-central1/models/724187836076523520 to Endpoint : projects/41965541199/locations/us-central1/endpoints/8783753203209469952
Deploy Endpoint model backing LRO: projects/41965541199/locations/us-central1/endpoints/8783753203209469952/operations/4965591770294386688
Endpoint model deployed. Resource name: projects/41965541199/locations/us-central1/endpoints/8783753203209469952


#using endpoint

In [25]:
## predict your data with online prediction here 
PROJECT_ID = 'dtidsus'
ENDPOINT_ID = "projects/41965541199/locations/us-central1/endpoints/8783753203209469952"
REGION = 'us-central1'



In [44]:
aiplatform.init(project=PROJECT_ID, location=REGION)
endpoint = aiplatform.Endpoint(ENDPOINT_ID)
prediction = endpoint.predict(instances=[[1, 1, 0, 0, 0, 1, 0, 0, 0, 60]])

print("PREDICTION:", prediction)

PREDICTION: Prediction(predictions=[0.0], deployed_model_id='4624375482123550720', metadata=None, model_version_id='1', model_resource_name='projects/41965541199/locations/us-central1/models/724187836076523520', explanations=None)


#manual file

In [28]:
try:
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name)
    blob_model = bucket.blob('ilham/model/model.pkl')
    blob_model.download_to_filename('model_new.pkl')

    print ("download model succeeded")
except:
    raise TypeError("An exception occurred")

download model succeeded


In [29]:
model_load = pickle.load(open('model_new.pkl','rb'))

In [30]:
y_predict = model_load.predict(X_test)

In [31]:
y_predict

array([0., 0., 0., ..., 0., 0., 1.])